Tabela Clientes

In [0]:
spark.sql("USE CATALOG trabalho_etl_bd3")
spark.sql("USE SCHEMA bronze")

df_clientes = spark.read.table("trabalho_etl_bd3.bronze.clientes")

df_clientes.show(5)


+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|id_cliente|                nome| uf|           cidade|      cep|sexo|faixa_etaria|data_cadastro|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|      C000|        otávio silva| RJ|         da Cunha|49167-377|   F|       18-25|   2009-10-20|
|      C001|        Bruna Aragão| ES|           Duarte| 85544678|   F|       18-25|   1997-03-25|
|      C002|     Alícia Oliveira| PE|  da Paz da Serra|43114-559|   F|       18-25|   2002-02-21|
|      C003|luiz fernando da ...| PB|           Castro|65883-297|   F|       36-45|   2000-08-28|
|      C004|      ISABELLA PINTO| AM|Martins de Farias|28100-185|   M|       18-25|   1977-11-02|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col, regexp_replace, trim, initcap, when, length, lower,to_timestamp, date_format, coalesce, expr

In [0]:
df_clientes = (
    df_clientes
    .withColumn("nome", regexp_replace(col("nome"), r"(?i)^\s*(sr|sra|srta|dr|dra|prof|profa)\.?\s+", ""))
    .withColumn("nome", regexp_replace(col("nome"), r"\s+", " "))
    .withColumn("nome", trim(col("nome")))
    .withColumn("nome", initcap(col("nome")))
)

df_clientes.show(50)

+----------+--------------------+---+--------------------+---------+----+------------+-------------+
|id_cliente|                nome| uf|              cidade|      cep|sexo|faixa_etaria|data_cadastro|
+----------+--------------------+---+--------------------+---------+----+------------+-------------+
|      C000|        Otávio Silva| RJ|            da Cunha|49167-377|   F|       18-25|   2009-10-20|
|      C001|        Bruna Aragão| ES|              Duarte| 85544678|   F|       18-25|   1997-03-25|
|      C002|     Alícia Oliveira| PE|     da Paz da Serra|43114-559|   F|       18-25|   2002-02-21|
|      C003|Luiz Fernando Da ...| PB|              Castro|65883-297|   F|       36-45|   2000-08-28|
|      C004|      Isabella Pinto| AM|   Martins de Farias|28100-185|   M|       18-25|   1977-11-02|
|      C005|      Thiago Da Mota| RR|              Aragão| 78984038|   M|       26-35|   1987-05-07|
|      C006|        Alexia Nunes| GO|   da Cunha da Prata|54181-146|   F|       18-25|   20

In [0]:
df_clientes = (
    df_clientes
    .withColumn("cep", col("cep").cast("string"))                 
    .withColumn("cep", regexp_replace(col("cep"), r"\D", ""))     
    .withColumn(
        "cep",
        when(length(col("cep")) == 8,
             
             regexp_replace(col("cep"), r"(\d{5})(\d{3})", "$1-$2")
        ).otherwise(col("cep"))
    )
)

df_clientes.show(5)

+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|id_cliente|                nome| uf|           cidade|      cep|sexo|faixa_etaria|data_cadastro|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|      C000|        Otávio Silva| RJ|         da Cunha|49167-377|   F|       18-25|   2009-10-20|
|      C001|        Bruna Aragão| ES|           Duarte|85544-678|   F|       18-25|   1997-03-25|
|      C002|     Alícia Oliveira| PE|  da Paz da Serra|43114-559|   F|       18-25|   2002-02-21|
|      C003|Luiz Fernando Da ...| PB|           Castro|65883-297|   F|       36-45|   2000-08-28|
|      C004|      Isabella Pinto| AM|Martins de Farias|28100-185|   M|       18-25|   1977-11-02|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
only showing top 5 rows


In [0]:
df_clientes = (
    df_clientes
    .withColumn("faixa_etaria", regexp_replace(col("faixa_etaria"), "_", "-"))
    .withColumn("faixa_etaria", lower(col("faixa_etaria")))
)

df_clientes.show(5)

+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|id_cliente|                nome| uf|           cidade|      cep|sexo|faixa_etaria|data_cadastro|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
|      C000|        Otávio Silva| RJ|         da Cunha|49167-377|   F|       18-25|   2009-10-20|
|      C001|        Bruna Aragão| ES|           Duarte|85544-678|   F|       18-25|   1997-03-25|
|      C002|     Alícia Oliveira| PE|  da Paz da Serra|43114-559|   F|       18-25|   2002-02-21|
|      C003|Luiz Fernando Da ...| PB|           Castro|65883-297|   F|       36-45|   2000-08-28|
|      C004|      Isabella Pinto| AM|Martins de Farias|28100-185|   M|       18-25|   1977-11-02|
+----------+--------------------+---+-----------------+---------+----+------------+-------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col, when, trim, lower

# if "sexo" not in df_clientes.columns:
#     raise ValueError(f"Coluna 'sexo' não existe. Colunas: {df_clientes.columns}")

df_clientes = (
    df_clientes
    .withColumn("sexo", lower(trim(col("sexo"))))
    .withColumn(
        "sexo",
        when(col("sexo").isin("f", "fem", "feminino"), "Feminino")
        .when(col("sexo").isin("m", "masc", "masculino"), "Masculino")
        .when((col("sexo") == "") | col("sexo").isNull(), "Não informado")
        .otherwise(col("sexo"))
    )
)

df_clientes.show(31)


+----------+--------------------+---+------------------+---------+-------------+------------+-------------+
|id_cliente|                nome| uf|            cidade|      cep|         sexo|faixa_etaria|data_cadastro|
+----------+--------------------+---+------------------+---------+-------------+------------+-------------+
|      C000|        Otávio Silva| RJ|          da Cunha|49167-377|     Feminino|       18-25|   2009-10-20|
|      C001|        Bruna Aragão| ES|            Duarte|85544-678|     Feminino|       18-25|   1997-03-25|
|      C002|     Alícia Oliveira| PE|   da Paz da Serra|43114-559|     Feminino|       18-25|   2002-02-21|
|      C003|Luiz Fernando Da ...| PB|            Castro|65883-297|     Feminino|       36-45|   2000-08-28|
|      C004|      Isabella Pinto| AM| Martins de Farias|28100-185|    Masculino|       18-25|   1977-11-02|
|      C005|      Thiago Da Mota| RR|            Aragão|78984-038|    Masculino|       26-35|   1987-05-07|
|      C006|        Alexia N

Tabela Entregas

In [0]:
df_entregas = spark.read.table("trabalho_etl_bd3.bronze.entregas")

df_entregas.show(5)

+---------+------------------+-----+-------------------+-----------------+-------------+
|id_pedido|parceiro_logistico|modal|janela_prevista_min|tempo_entrega_min|atrasado_flag|
+---------+------------------+-----+-------------------+-----------------+-------------+
|     1101|             Loggi| bike|                 20|               38|            1|
|     1027|             Loggi| bike|                 20|               14|            0|
|     1052|             Rappi|carro|                 25|               42|            1|
|     1080|             Rappi| bike|                 15|               34|            1|
|     1078|           Interno| bike|                 15|               20|            1|
+---------+------------------+-----+-------------------+-----------------+-------------+
only showing top 5 rows


In [0]:
df_entregas = df_entregas.withColumn(
    "parceiro_logistico",
    when(
        (col("parceiro_logistico").isNull()) | (trim(col("parceiro_logistico")) == ""),
        "nao informado"
    ).otherwise(trim(col("parceiro_logistico")))
)

df_entregas.show(15)

+---------+------------------+-----+-------------------+-----------------+-------------+
|id_pedido|parceiro_logistico|modal|janela_prevista_min|tempo_entrega_min|atrasado_flag|
+---------+------------------+-----+-------------------+-----------------+-------------+
|     1101|             Loggi| bike|                 20|               38|            1|
|     1027|             Loggi| bike|                 20|               14|            0|
|     1052|             Rappi|carro|                 25|               42|            1|
|     1080|             Rappi| bike|                 15|               34|            1|
|     1078|           Interno| bike|                 15|               20|            1|
|     1114|             Rappi|carro|                 15|               28|            1|
|     1037|             Loggi|carro|                 20|               40|            1|
|     1106|             Flash|drone|                 20|               14|            0|
|     1086|          

In [0]:
df_entregas = df_entregas.filter(
    (col("janela_prevista_min") > 0) &
    (col("tempo_entrega_min") > 0)
)

df_entregas.show(15)

+---------+------------------+-----+-------------------+-----------------+-------------+
|id_pedido|parceiro_logistico|modal|janela_prevista_min|tempo_entrega_min|atrasado_flag|
+---------+------------------+-----+-------------------+-----------------+-------------+
|     1101|             Loggi| bike|                 20|               38|            1|
|     1027|             Loggi| bike|                 20|               14|            0|
|     1052|             Rappi|carro|                 25|               42|            1|
|     1080|             Rappi| bike|                 15|               34|            1|
|     1078|           Interno| bike|                 15|               20|            1|
|     1114|             Rappi|carro|                 15|               28|            1|
|     1037|             Loggi|carro|                 20|               40|            1|
|     1106|             Flash|drone|                 20|               14|            0|
|     1086|          

In [0]:
df_entregas = df_entregas.withColumn(
    "atrasado_flag",
    when(lower(trim(col("atrasado_flag"))) == "yes", "teve atraso")
    .when(col("atrasado_flag") == 1, "teve atraso")
    .when(col("atrasado_flag") == 0, "não teve atraso")
    .otherwise("não teve atraso")  # padrão de segurança
)

df_entregas.show(5)

+---------+------------------+-----+-------------------+-----------------+---------------+
|id_pedido|parceiro_logistico|modal|janela_prevista_min|tempo_entrega_min|  atrasado_flag|
+---------+------------------+-----+-------------------+-----------------+---------------+
|     1101|             Loggi| bike|                 20|               38|    teve atraso|
|     1027|             Loggi| bike|                 20|               14|não teve atraso|
|     1052|             Rappi|carro|                 25|               42|    teve atraso|
|     1080|             Rappi| bike|                 15|               34|    teve atraso|
|     1078|           Interno| bike|                 15|               20|    teve atraso|
+---------+------------------+-----+-------------------+-----------------+---------------+
only showing top 5 rows


Tabela Pedidos

In [0]:
df_pedidos = spark.read.table("trabalho_etl_bd3.bronze.pedidos")

df_pedidos.show(5)

+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
|     1001|         1|09/06/2025 17:10|     157.06|        2|         1|               15|             3|
|     1002|        38|2025-06-07T12:36|     -19.87|        5|         1|               15|             3|
|     1003|        39|2025-06-04T09:13|     150.89|        5|         0|               25|             5|
|     1004|         9|2025-09-27T18:48|     114.36|        4|         1|               20|             5|
|     1005|        27|2025-07-21T06:37|      29.27|        6|         1|               35|             5|
+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
only showing top 5 rows


In [0]:
df_pedidos = df_pedidos.filter(col("valor_total").cast("double") > 0)
df_pedidos.show(5)


+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
|     1001|         1|09/06/2025 17:10|     157.06|        2|         1|               15|             3|
|     1003|        39|2025-06-04T09:13|     150.89|        5|         0|               25|             5|
|     1004|         9|2025-09-27T18:48|     114.36|        4|         1|               20|             5|
|     1005|        27|2025-07-21T06:37|      29.27|        6|         1|               35|             5|
|     1006|        32|2025-06-25T11:10|      54.21|        6|         0|               15|             3|
+---------+----------+----------------+-----------+---------+----------+-----------------+--------------+
only showing top 5 rows


In [0]:
df_pedidos = df_pedidos.withColumn(
    "usou_cupom",
    when(lower(trim(col("usou_cupom"))).isin("0", "false", "f", "nao", "não"), "nao utilizou")
    .when(lower(trim(col("usou_cupom"))).isin("1", "true", "t", "sim", "yes"), "utilizou")
    .otherwise("nao utilizou")
)
df_pedidos.show(5)



+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|  usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|     1001|         1|09/06/2025 17:10|     157.06|        2|    utilizou|               15|             3|
|     1003|        39|2025-06-04T09:13|     150.89|        5|nao utilizou|               25|             5|
|     1004|         9|2025-09-27T18:48|     114.36|        4|    utilizou|               20|             5|
|     1005|        27|2025-07-21T06:37|      29.27|        6|    utilizou|               35|             5|
|     1006|        32|2025-06-25T11:10|      54.21|        6|nao utilizou|               15|             3|
+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
only showing top 5 rows


In [0]:
df_pedidos = df_pedidos.filter(col("tempo_preparo_min").cast("int") > 0)
df_pedidos.show(15)



+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|  usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|     1001|         1|09/06/2025 17:10|     157.06|        2|    utilizou|               15|             3|
|     1003|        39|2025-06-04T09:13|     150.89|        5|nao utilizou|               25|             5|
|     1004|         9|2025-09-27T18:48|     114.36|        4|    utilizou|               20|             5|
|     1005|        27|2025-07-21T06:37|      29.27|        6|    utilizou|               35|             5|
|     1006|        32|2025-06-25T11:10|      54.21|        6|nao utilizou|               15|             3|
|     1007|        25|2025-09-11T22:15|     177.72|        1|    utilizou|               30|             5|
|     1008|        34|2025-0

In [0]:
df_pedidos = df_pedidos.withColumn(
    "nota_avaliacao",
    when(col("nota_avaliacao").isNull() | (trim(col("nota_avaliacao")) == ""), "nao informado")
    .otherwise(col("nota_avaliacao"))
)
df_pedidos.show(15)



+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|  usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|     1001|         1|09/06/2025 17:10|     157.06|        2|    utilizou|               15|             3|
|     1003|        39|2025-06-04T09:13|     150.89|        5|nao utilizou|               25|             5|
|     1004|         9|2025-09-27T18:48|     114.36|        4|    utilizou|               20|             5|
|     1005|        27|2025-07-21T06:37|      29.27|        6|    utilizou|               35|             5|
|     1006|        32|2025-06-25T11:10|      54.21|        6|nao utilizou|               15|             3|
|     1007|        25|2025-09-11T22:15|     177.72|        1|    utilizou|               30|             5|
|     1008|        34|2025-0

In [0]:
from pyspark.sql.functions import col, regexp_replace, trim, coalesce, date_format, expr

df_pedidos = df_pedidos.withColumn(
    "data_pedido_norm",
    trim(
        regexp_replace(
            regexp_replace(col("data_pedido").cast("string"), "T", " "),
            "/", "-"
        )
    )
)

df_pedidos = df_pedidos.withColumn(
    "data_pedido_ts",
    coalesce(
        expr("try_to_timestamp(data_pedido_norm, 'dd-MM-yyyy HH:mm')"),
        expr("try_to_timestamp(data_pedido_norm, 'yyyy-MM-dd HH:mm')"),
        expr("try_to_timestamp(data_pedido_norm, 'yyyy-MM-dd HH:mm:ss')"),
        expr("try_to_timestamp(data_pedido_norm, 'dd-MM-yyyy HH:mm:ss')")
    )
)

df_pedidos = (
    df_pedidos
    .withColumn("data_pedido", date_format(col("data_pedido_ts"), "yyyy-MM-dd HH:mm"))
    .drop("data_pedido_norm", "data_pedido_ts")
)

# 4️⃣ conferir resultado
df_pedidos.show(10)


+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|id_pedido|id_cliente|     data_pedido|valor_total|qtd_itens|  usou_cupom|tempo_preparo_min|nota_avaliacao|
+---------+----------+----------------+-----------+---------+------------+-----------------+--------------+
|     1001|         1|2025-06-09 17:10|     157.06|        2|    utilizou|               15|             3|
|     1003|        39|2025-06-04 09:13|     150.89|        5|nao utilizou|               25|             5|
|     1004|         9|2025-09-27 18:48|     114.36|        4|    utilizou|               20|             5|
|     1005|        27|2025-07-21 06:37|      29.27|        6|    utilizou|               35|             5|
|     1006|        32|2025-06-25 11:10|      54.21|        6|nao utilizou|               15|             3|
|     1007|        25|2025-09-11 22:15|     177.72|        1|    utilizou|               30|             5|
|     1008|        34|2025-0

In [0]:
df_clientes.filter(df_clientes.uf.isNull()).show()


+----------+----+---+------+---+----+------------+-------------+
|id_cliente|nome| uf|cidade|cep|sexo|faixa_etaria|data_cadastro|
+----------+----+---+------+---+----+------------+-------------+
+----------+----+---+------+---+----+------------+-------------+



In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS trabalho_etl_bd3.silver")
spark.sql("USE SCHEMA silver")

df_clientes.write.format("delta").mode("overwrite")\
    .saveAsTable("trabalho_etl_bd3.silver.clientes")

df_pedidos.write.format("delta").mode("overwrite")\
    .saveAsTable("trabalho_etl_bd3.silver.pedidos")

df_entregas.write.format("delta").mode("overwrite")\
    .saveAsTable("trabalho_etl_bd3.silver.entregas")
